In [3]:
from fastapi import FastAPI, HTTPException
import requests
import os

app = FastAPI()

# Instagram API Credentials (Replace with actual tokens)
INSTAGRAM_ACCESS_TOKEN = os.getenv("INSTAGRAM_ACCESS_TOKEN")
INSTAGRAM_API_URL = "https://graph.facebook.com/v18.0"

@app.get("/fetch_engagement/{user_id}")
def fetch_engagement(user_id: str):
    """
    Fetch Instagram engagement metrics for a given user ID.
    """
    url = f"{INSTAGRAM_API_URL}/{user_id}/media?fields=id,caption,like_count,comments_count,insights.metric(engagement,impressions,reach,saved,video_views)&access_token={INSTAGRAM_ACCESS_TOKEN}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Failed to fetch data")
    
    return response.json()

@app.get("/detect_viral/{user_id}")
def detect_viral(user_id: str):
    """
    Analyze engagement data and detect if content is going viral.
    """
    data = fetch_engagement(user_id)
    
    viral_posts = []
    for post in data.get("data", []):
        if "insights" in post:
            metrics = post["insights"]
            if metrics["engagement"] > 100000 or metrics["video_views"] > 1000000:
                viral_posts.append(post)
    
    return {"viral_posts": viral_posts}


In [9]:
from fastapi import FastAPI, HTTPException
import requests
import os
import mysql.connector

app = FastAPI()

# Instagram API Credentials (Replace with actual tokens)
INSTAGRAM_ACCESS_TOKEN = os.getenv("INSTAGRAM_ACCESS_TOKEN")
INSTAGRAM_API_URL = "https://graph.facebook.com/v18.0"

# Database connection
DATABASE_CONFIG = {
    "host": os.getenv("DB_HOST", "localhost"),
    "user": os.getenv("DB_USER", "root"),
    "password": os.getenv("DB_PASSWORD", ""),
    "database": os.getenv("DB_NAME", "instagram_db")
}

conn = mysql.connector.connect(**DATABASE_CONFIG)
cursor = conn.cursor(dictionary=True)

# Create table if not exists
cursor.execute('''
    CREATE TABLE IF NOT EXISTS engagement (
        id INT AUTO_INCREMENT PRIMARY KEY,
        user_id VARCHAR(255),
        post_id VARCHAR(255),
        caption TEXT,
        like_count INT,
        comments_count INT,
        engagement INT,
        impressions INT,
        reach INT,
        saved INT,
        video_views INT,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
''')
conn.commit()

@app.get("/fetch_engagement/{user_id}")
def fetch_engagement(user_id: str):
    """
    Fetch Instagram engagement metrics for a given user ID and store in database.
    """
    url = f"{INSTAGRAM_API_URL}/{user_id}/media?fields=id,caption,like_count,comments_count,insights.metric(engagement,impressions,reach,saved,video_views)&access_token={INSTAGRAM_ACCESS_TOKEN}"
    
    response = requests.get(url)
    if response.status_code != 200:
        raise HTTPException(status_code=response.status_code, detail="Failed to fetch data")
    
    data = response.json()
    for post in data.get("data", []):
        if "insights" in post:
            metrics = post["insights"]
            cursor.execute('''
                INSERT INTO engagement (user_id, post_id, caption, like_count, comments_count, engagement, impressions, reach, saved, video_views)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            ''', (user_id, post["id"], post.get("caption", ""), post["like_count"], post["comments_count"], 
                  metrics["engagement"], metrics["impressions"], metrics["reach"], metrics["saved"], metrics["video_views"]))
            conn.commit()
    
    return data

@app.get("/detect_viral/{user_id}")
def detect_viral(user_id: str):
    """
    Analyze engagement data and detect if content is going viral.
    """
    cursor.execute('''
        SELECT * FROM engagement WHERE user_id = %s AND (engagement > 100000 OR video_views > 1000000)
    ''', (user_id,))
    viral_posts = cursor.fetchall()
    
    return {"viral_posts": viral_posts}


ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)

In [10]:
print("DB_HOST:", os.getenv("DB_HOST"))
print("DB_USER:", os.getenv("DB_USER"))
print("DB_PASSWORD:", os.getenv("DB_PASSWORD"))
print("DB_NAME:", os.getenv("DB_NAME"))


DB_HOST: None
DB_USER: None
DB_PASSWORD: None
DB_NAME: None
